In [1]:
import pandas as pd

#Dataframes for each CSV File

df_agencies= pd.read_csv("agencies.csv") 
df_act_type = pd.read_csv("NIBRS_CRIMINAL_ACT_TYPE.csv")
df_act = pd.read_csv("NIBRS_CRIMINAL_ACT.csv")
df_incident = pd.read_csv("NIBRS_incident.csv")
df_location = pd.read_csv("NIBRS_LOCATION_TYPE.csv")
df_month = pd.read_csv("NIBRS_month.csv")
df_offense_type = pd.read_csv("NIBRS_OFFENSE_TYPE.csv")
df_offense = pd.read_csv("NIBRS_OFFENSE.csv")

df = df_incident\
    .merge(df_agencies, on="agency_id", how="outer")\
    .merge(df_month, on=["agency_id", "nibrs_month_id", "did"], how="outer")\
    .merge(df_offense, on="incident_id", how="outer", suffixes=("_left", "_right"))\
    .merge(df_offense_type, on="offense_code", how="outer")\
    .merge(df_act, on="offense_id", how="outer")\
    .merge(df_act_type, on="criminal_act_id", how="outer")\
    .merge(df_location, on="location_id", how = "outer")

df.to_csv("2024_merged_data.csv", index=False)

##### Removing Duplicates (If Any)

In [2]:
column_names = df.columns.tolist()
print("Number of Rows before removing duplicates: ", len(df))
df.drop_duplicates(subset=column_names, keep='first', inplace=True)
print('Number of Rows after removing Duplicates:', len(df))

Number of Rows before removing duplicates:  118796
Number of Rows after removing Duplicates: 118796


##### Removing Columns with Nulls or No Purpose

In [3]:
test = df.isna().sum()
for col, count in test.items():
    if count != 0:
        print(col, count)

data_year_x 40
agency_id 39
incident_id 40
nibrs_month_id 40
cargo_theft_flag 40
submission_date 40
incident_date 40
report_date_flag 40
incident_hour 335
cleared_except_id 40
cleared_except_date 118718
incident_status 40
data_home_x 118796
orig_format_x 40
did 40
yearly_agency_id 39
data_year_y 39
ori 39
legacy_ori 39
covered_by_legacy_ori 118796
direct_contributor_flag 39
dormant_flag 39
dormant_year 118796
reporting_type 39
ucr_agency_name 39
ncic_agency_name 785
pub_agency_name 39
pub_agency_unit 118530
agency_status 39
state_id 39
state_name 39
state_abbr 39
state_postal_abbr 39
division_code 39
division_name 39
region_code 39
region_name 39
region_desc 39
agency_type_name 39
population 39
submitting_agency_id 39
sai 39
submitting_agency_name 39
suburban_area_flag 39
population_group_id 39
population_group_code 39
population_group_desc 39
parent_pop_group_code 39
parent_pop_group_desc 39
mip_flag 39
pop_sort_order 39
summary_rape_def 118796
pe_reported_flag 39
male_officer 39
male

In [4]:
#These Columns had Null Entries
df.drop(columns=["cleared_except_date", 'data_home_x', 'covered_by_legacy_ori', 'dormant_year',\
                 'pub_agency_unit', 'report_date', 'update_flag', 'data_home_y', 'month_pub_status',\
                 'num_premises_entered', 'summary_rape_def', 'method_entry_code'], inplace=True)

#Manually Inspected The CSV
df.drop(columns=['nibrs_month_id_x', 'cargo_theft_flag', 'submission_date', 'report_date_flag', 'incident_id',\
                 'cleared_except_id', 'incident_status', 'orig_format_x', 'did', 'data_year_y', 'legacy_ori',\
                 'direct_contributor_flag', 'dormant_flag', 'reporting_type', 'ucr_agency_name', 'nibrs_month_id_x',\
                 'agency_status', 'state_id', 'state_abbr', 'state_postal_abbr', 'division_code', 'division_name',\
                 'region_code', 'region_name', 'region_desc', 'agency_type_name', 'submitting_agency_id', 'sai', 'ori',\
                 'submitting_agency_name', 'suburban_area_flag', 'population_group_id', 'population_group_code', 'yearly_agency_id',\
                 'population_group_desc', 'parent_pop_group_code', 'parent_pop_group_desc', 'mip_flag', 'pop_sort_order',\
                 'pe_reported_flag', 'officer_rate', 'employee_rate', 'nibrs_cert_date', 'nibrs_start_date', 'nibrs_leoka_start_date',\
                 'nibrs_ct_start_date',	'nibrs_multi_bias_start_date', 'nibrs_off_eth_start_date', 'covered_flag', 'county_name',\
                 'msa_name', 'publishable_flag', 'participated', 'nibrs_participated', 'data_year_left', 'nibrs_month_id_y',\
                 'agency_id_y',	'month_num', 'inc_data_year', 'reported_status', 'orig_format_y', 'ddocname', 'data_year_right',\
                 'offense_id', 'offense_code', 'attempt_complete_flag', 'location_id', 'ct_flag', 'hc_flag', 'hc_code', 'crime_against', \
                 'offense_group', 'data_year', 'criminal_act_id', 'criminal_act_code', 'criminal_act_desc', 'ncic_agency_name', 'location_code'],
                 inplace = True, errors='ignore')

df.rename(columns={'data_year_x': 'year'}, inplace=True)
df.rename(columns={'agency_id_x': 'agency_location'}, inplace=True)
df.rename(columns={'pub_agency_name': 'agency_name'}, inplace=True)
df.rename(columns={'state_name': 'state'}, inplace=True)
df.rename(columns={'location_name': 'location_area'}, inplace=True)

#ncic_agency_name


##### Check Datatype for each Variable. Change Date Datatype.

In [5]:
df.info()
df['incident_date'] = pd.to_datetime(df['incident_date'], format='%Y-%m-%d', errors='coerce')
print("\n")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118796 entries, 0 to 118795
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   year                            118756 non-null  float64
 1   agency_id                       118757 non-null  float64
 2   nibrs_month_id                  118756 non-null  float64
 3   incident_date                   118756 non-null  object 
 4   incident_hour                   118461 non-null  float64
 5   agency_name                     118757 non-null  object 
 6   state                           118757 non-null  object 
 7   population                      118757 non-null  float64
 8   male_officer                    118757 non-null  float64
 9   male_civilian                   118757 non-null  float64
 10  male_officer+male_civilian      118757 non-null  float64
 11  female_officer                  118757 non-null  float64
 12  female_civilian 

In [6]:
df.drop(columns=['nibrs_month_id', 'nibrs_month_id', 'state', 'criminal_act_name'], inplace=True)
df.dropna(inplace=True)
df['population'] = df['population'].astype(int)
df['year'] = df['year'].astype(int)
df['incident_hour'] = df['incident_hour'].astype(int)



In [7]:
df.to_csv("2024_merged_data.csv", index=False)

In [8]:
df.isna().sum()


year                              0
agency_id                         0
incident_date                     0
incident_hour                     0
agency_name                       0
population                        0
male_officer                      0
male_civilian                     0
male_officer+male_civilian        0
female_officer                    0
female_civilian                   0
female_officer+female_civilian    0
offense_name                      0
offense_category_name             0
location_area                     0
dtype: int64

In [9]:
df

,year,agency_id,incident_date,incident_hour,agency_name,population,male_officer,male_civilian,male_officer+male_civilian,female_officer,female_civilian,female_officer+female_civilian,offense_name,offense_category_name,location_area
0,2024,2159.0,2024-04-26,18,Meriden,59943,104.0,4.0,108.0,20.0,8.0,28.0,Drug/Narcotic Violations,Drug/Narcotic Offenses,Abandoned/Condemned Structure
1,2024,2159.0,2024-10-17,10,Meriden,59943,104.0,4.0,108.0,20.0,8.0,28.0,Drug Equipment Violations,Drug/Narcotic Offenses,Abandoned/Condemned Structure
2,2024,2135.0,2024-10-05,17,Danbury,86257,142.0,0.0,142.0,14.0,8.0,22.0,Drug/Narcotic Violations,Drug/Narcotic Offenses,Abandoned/Condemned Structure
3,2024,2135.0,2024-10-05,17,Danbury,86257,142.0,0.0,142.0,14.0,8.0,22.0,Drug Equipment Violations,Drug/Narcotic Offenses,Abandoned/Condemned Structure
4,2024,2135.0,2024-10-05,17,Danbury,86257,142.0,0.0,142.0,14.0,8.0,22.0,Drug/Narcotic Violations,Drug/Narcotic Offenses,Abandoned/Condemned Structure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118751,2024,2168.0,2024-07-21,3,New Haven,137243,279.0,12.0,291.0,52.0,38.0,90.0,Aggravated Assault,Assault Offenses,Other/Unknown
118752,2024,2200.0,2024-12-12,9,Torrington,35612,63.0,2.0,65.0,7.0,9.0,16.0,Intimidation,Assault Offenses,Other/Unknown
118753,2024,2168.0,2024-11-21,15,New Haven,137243,279.0,12.0,291.0,52.0,38.0,90.0,All Other Larceny,Larceny/Theft Offenses,Other/Unknown
118754,2024,2168.0,2024-12-05,22,New Haven,137243,279.0,12.0,291.0,52.0,38.0,90.0,Robbery,Robbery,Other/Unknown
